# Project:  Traffic Sign Classification 
**In this project, We used Python and TensorFlow to classify traffic signs.**

**Dataset used: [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).
This dataset has more than 50,000 images of 43 classes.**

**We were able to reach a 95% testing accuracy.**

In [ ]:
import tensorflow as tf
from tensorflow import keras 
from PIL import Image, ImageOps
import cv2 
import random
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt

from keras.callbacks import ModelCheckpoint, EarlyStopping
import time
from tqdm import tqdm
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from sklearn.metrics import confusion_matrix


## Loading train data

In [ ]:
df = pd.read_csv("Train.csv")
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)  #for train


In [ ]:
df.info()

In [ ]:
for i, row in df.iterrows():
    print(row['Path'])

In [ ]:
data=[]
labels=[]


In [ ]:
for i,row  in tqdm(df.iterrows()):
    
    # Replace this with the path to your image
    image = Image.open(row['Path'])

    #resize the image to a 30x30 with the same strategy as in TM2:
    image=image.resize((30,30))

    #turn the image into a numpy array
    image=np.array(image)
    

    # Normalize the image
    #normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    # Load the image into the array
    data.append(image)
    labels.append(row['ClassId'])
    

In [ ]:
data = np.array(data)
data.shape  

In [ ]:
labels=np.array(labels)
labels.shape

# Testing the data with original test data
---
## Loading Test Data

In [ ]:
df_new = pd.read_csv("Test.csv")
data_new = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

In [ ]:
df_new.head()   

In [ ]:
for i, row in df_new.iterrows():
    print(row['Path'])

In [ ]:
data_new=[]
labels_new=[]

In [ ]:
for i,row  in tqdm(df_new.iterrows()):
    
    # Replace this with the path to your image
    image_new = Image.open(row['Path'])

    #resize the image to a 30x30 with the same strategy as in TM2:
    image_new=image_new.resize((30,30))

    #turn the image into a numpy array
    #image_new=np.array(image_new)
    image_new = np.asarray(image_new, dtype=np.uint8)
    
    #converting the array into integers
    #image_new = image_new.astype('uint8')

    # Normalize the image
    #normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    # Load the image into the array
    data_new.append(image_new)
    labels_new.append(row['ClassId'])
    

In [ ]:
data_new = np.array(data_new)
data_new.shape  

In [ ]:
labels_new=np.array(labels_new)
labels_new.shape

In [ ]:
X_train_new = data
y_train_new = labels
X_test_new = data_new
y_test_new = labels_new

In [ ]:
print(X_train_new.shape,y_train_new.shape,X_test_new.shape,y_test_new.shape)

## **Grayscaling**:This is used to improve test accuracy

In [ ]:
#converting to grayscale
def list_images(data,labels, ylabel="", cmap=None):
    """
    Display a list of images in a single figure with matplotlib.
        Parameters:
            images: An np.array compatible with plt.imshow.
            lanel (Default = No label): A string to be used as a label for each image.
            cmap (Default = None): Used to display gray images.
    """
    plt.figure(figsize=(15, 16))
    for i in range(6):
        plt.subplot(1, 6, i+1)
        indx = random.randint(0, len(data))
        #Use gray scale color map if there is only one channel
        cmap = 'gray' if len(data[indx].shape) == 2 else cmap
        plt.imshow(data[indx], cmap = cmap)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()

## **Grayscaling** the train data

In [ ]:
 def gray_scale(data):                                              #train data
    """                                                            
    Convert images to gray scale.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    return cv2.cvtColor(data, cv2.COLOR_RGB2GRAY)

In [ ]:
# Sample images after grayscaling
gray_images = list(map(gray_scale, X_train_new))
list_images(gray_images, y_train_new, "Gray Scale image", "gray")


In [ ]:
g_img = np.array(gray_images)

In [ ]:
g_img = g_img.reshape(39209,30,30,1)

In [ ]:
def list_images_new(data_new,labels_new, ylabel="", cmap=None):
    """
    Display a list of images in a single figure with matplotlib.
        Parameters:
            images: An np.array compatible with plt.imshow.
            lanel (Default = No label): A string to be used as a label for each image.
            cmap (Default = None): Used to display gray images.
    """
    plt.figure(figsize=(15, 16))
    for i in range(6):
        plt.subplot(1, 6, i+1)
        indx = random.randint(0, len(data_new))
        #Use gray scale color map if there is only one channel
        cmap = 'gray' if len(data_new[indx].shape) == 2 else cmap
        plt.imshow(data_new[indx], cmap = cmap)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()

## **Grayscaling** the test data

In [ ]:

def gray_scale_new(data_new):                                              
    return cv2.cvtColor(data_new, cv2.COLOR_RGB2GRAY)



In [ ]:
# Sample images after grayscaling
gray_images_new = list(map(gray_scale_new, X_test_new))
list_images_new(gray_images_new, y_test_new, "Gray Scale image", "gray")

In [ ]:
g_img_new = np.array(gray_images_new)

In [ ]:
g_img_new = g_img_new.reshape(12630,30,30,1)

## Creating the model

In [ ]:
def get_conv_model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation ='relu', input_shape = input_shape))
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation ='relu'))
    model.add(MaxPool2D(pool_size = (2,2) ))
    model.add(Dropout(rate=0.25))
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation ='relu'))
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation ='relu'))
    model.add(MaxPool2D(pool_size = (2,2) ))
    model.add(Dropout(rate=0.25))
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(43, activation = 'softmax'))
    model.summary()
    
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])
    return model
    

In [ ]:
input_shape = g_img[0].shape


## Fitting the model

In [ ]:
#model.callback
model_path = 'traffic_sign_model_grayscale_2'
epochs = 20
batch_size =256

model = get_conv_model()
es = EarlyStopping(monitor = 'val_loss', verbose =1, mode =min, patience = 3)  
checkpoint = ModelCheckpoint(model_path, monitor = 'acc', verbose = 1, mode='max',
                             save_best_only=True, save_weights_only = False, period =1)
history = model.fit(g_img, y_train_new, batch_size=batch_size, shuffle = True, epochs= epochs, 
                    verbose = 1, validation_split=0.05, callbacks = [checkpoint])

model.save(model_path)         

## Getting the final test score(ie Accuracy score)

In [ ]:

yp=[]
print(y_test_new)


model=load_model(model_path)
y_pred = model.predict(g_img_new)
print(y_pred.shape)

for i in y_pred:
    vec=list(i)
    yp.append(vec.index(max(vec)))
        
#print(yp)
#print(len(y_test))

from sklearn.metrics import accuracy_score

print('Accuracy score : ', accuracy_score(y_test_new,yp))

from sklearn.metrics import confusion_matrix

c = confusion_matrix(y_test_new,yp)
print(c)

from sklearn.metrics import classification_report
target_names = []
for i in range(43):
    target_names.append('class '+str(i))
print((classification_report(y_test_new,yp,target_names = target_names)))


## Displaying the confusion matrix

In [ ]:
cm = confusion_matrix(y_test_new, yp)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm = np.log(.0001 + cm)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Log of normalized Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()